In [19]:
import math
import networkx as nx
import numpy as np
import pickle
import matplotlib.pyplot as plt
import random

In [32]:
def euclidean(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def generate_positions(graph, blockade=7.5):
    positions = {}
    placed_nodes = set()

    first_node = list(graph.keys())[0]
    positions[first_node] = (0, 0)
    placed_nodes.add(first_node)

    def find_position(node):
        candidate_position = None
        valid_position = False
        while not valid_position:
            angle = random.randrange(360)
            x = blockade*math.cos(math.radians(angle))
            y = blockade*math.sin(math.radians(angle))
            candidate_position = (x, y)
            valid_position = True
            for neighbor in placed_nodes:
                if neighbor in graph[node]:
                    dist = euclidean(candidate_position, positions[neighbor])
                    if dist > blockade or candidate_position in positions.values():
                        valid_position = False
                        break
                else:
                    dist = euclidean(candidate_position, positions[neighbor])
                    if dist <= blockade or candidate_position in positions.values():
                        valid_position = False
                        break
            if valid_position:
                return candidate_position

    for node in graph:
        if node not in placed_nodes:
            new_position = find_position(node)
            positions[node] = new_position
            placed_nodes.add(node)

    min_x = min(x for x, _ in positions.values())
    min_y = min(y for _, y in positions.values())

    if min_x < 0 or min_y < 0:
        shift_x = abs(min_x) if min_x < 0 else 0
        shift_y = abs(min_y) if min_y < 0 else 0
        positions = {node: (x + shift_x, y + shift_y) for node, (x, y) in positions.items()}

    return [i for i in positions.values()]

In [33]:
graph = {
    'A': {'B': 1, 'C': 1},
    'B': {'A': 1, 'C': 1, 'D': 1},
    'C': {'A': 1, 'B': 1, 'D': 1},
    'D': {'B': 1, 'C': 1, 'D': 1},
}
positions = generate_positions(graph)
print(positions)
with open("graph.pickle", "wb") as file:
    pickle.dump(positions, file)

[(0, 0), (3.6360721518475283, 6.559647803545468), (7.427010515561777, 1.0437982572004907), (4.408389392193548, 6.067627457812106)]
